In [1]:
#default_exp traindenovo

## De Novo Training Script

In [2]:
#export
import sys
sys.path.insert(0, "/home/turgutluk/Vent_Seg_Project/dev/")

In [3]:
# set just for notebook environemnt
import os; os.environ['YAML_PATH']="/home/turgutluk/Vent_Seg_Project/dev/data.yaml"

In [4]:
#export
from fastai2.vision.all import *
from fastai2.data.all import *
from local.datasource import *
from local.models import *
from fastai2.torch_core import *
from fastai2.basics import *
from local.trainutils import *
from fastai2.callback.all import *
from fastai2.distributed import *
from time import time

In [5]:
len(datasource_dict), datasource_dict # all possible dataset for experiments, (atlas, mr, ct) x (brain, ventricle)

(6,
 {'notl_brain_mr': functools.partial(<function create_mr_ct_dsource at 0x7f55cc3df320>, '/home/turgutluk/data/ventricles_data/mr', '/home/turgutluk/data/ventricles_data/csvs/mr_splits_df.csv', <function tfm_image_pt at 0x7f55cc3d9320>, <function tfm_brain_mask_pt at 0x7f564c38f950>),
  'notl_brain_ct': functools.partial(<function create_mr_ct_dsource at 0x7f55cc3df320>, '/home/turgutluk/data/ventricles_data/ct', '/home/turgutluk/data/ventricles_data/csvs/ct_splits_df.csv', <function tfm_image_pt at 0x7f55cc3d9320>, <function tfm_brain_mask_pt at 0x7f564c38f950>),
  'notl_ventricle_mr': functools.partial(<function create_mr_ct_dsource at 0x7f55cc3df320>, '/home/turgutluk/data/ventricles_data/mr', '/home/turgutluk/data/ventricles_data/csvs/mr_splits_df.csv', <function tfm_skull_stripped_image_pt at 0x7f55cc3df170>, <function tfm_ventricles_mask_pt at 0x7f55cc3df200>),
  'notl_ventricle_ct': functools.partial(<function create_mr_ct_dsource at 0x7f55cc3df320>, '/home/turgutluk/data/ven

In [6]:
len(experiment_model_dict), experiment_model_dict # all possible model configs for experiments

(11,
 {'baseline1': functools.partial(<function unet_default at 0x7f55cc3df0e0>, p=0.0, norm_type='batch', actn='relu'),
  'baseline2': functools.partial(<function unet_default at 0x7f55cc3df0e0>, p=0.0, norm_type='batch', actn='relu'),
  'baseline3': functools.partial(<function unet_default at 0x7f55cc3df0e0>, p=0.0, norm_type='group', actn='relu'),
  'baseline4': functools.partial(<function unet_default at 0x7f55cc3df0e0>, p=0.0, norm_type='group', actn='prelu'),
  'baseline5': functools.partial(<function unet_default at 0x7f55cc3df0e0>, p=0.3, norm_type='group', actn='prelu'),
  'baseline6': <function local.models.meshnet()>,
  'baseline7': functools.partial(<function unet_wide at 0x7f55cc3f4290>, p=0.0, norm_type='group', actn='prelu'),
  'baseline8': functools.partial(<function unet_deep at 0x7f55cc3f4320>, p=0.0, norm_type='group', actn='prelu'),
  'baseline9': functools.partial(<function unet_wide_deep at 0x7f55cc3f43b0>, p=0.0, norm_type='group', actn='prelu'),
  'baseline10': 

### args for experiments

In [7]:
gpu=0

In [75]:
data_name = "atlas_brain_mr"
sample_size = 10
seed=42
bs=2
model_name = 'baseline1'
MODEL_NAME='ATLAS_Brain_MR_Baseline_1'
model_dir='atlas_brain_mr_models'
early_stop=False
loss_func='bce'
eps=1e-8
epochs=1
lr=0.1

In [46]:
data_name = "notl_brain_mr"
sample_size = 10
seed=42
bs=2
model_name = 'baseline1'
MODEL_NAME='NOTL_Brain_MR_Baseline_1'
model_dir='notl_brain_mr_models'
early_stop=False
loss_func='bce'
eps=1e-8
epochs=1
lr=0.1

### 1) data

In [76]:
#export
@contextmanager
def np_local_seed(seed):
    "numpy local seed - doesn't effect global random state"
    state = np.random.get_state()
    np.random.seed(seed)
    try: yield
    finally: np.random.set_state(state)

In [77]:
dsource = datasource_dict[data_name]()

In [78]:
if sample_size:
    with np_local_seed(seed):
        dsource.splits[0] = L(np.random.choice(dsource.splits[0], sample_size))  # subsample training

In [79]:
dsource.splits[0]

array([ 860, 1294, 1130, 1095, 1638,  466, 1238,  330, 1482, 2135])

In [80]:
dbunch = dsource.databunch(after_batch=[Cuda()], bs=bs)

In [81]:
dbunch.dls

(<fastai2.data.core.TfmdDL at 0x7f55c24d5590>,
 <fastai2.data.core.TfmdDL at 0x7f55c239af10>)

In [82]:
# create for test1 and test2 dls
if len(dbunch.dls) == 4: pass
elif len(dbunch.dls) == 2: pass
else: raise Exception(f"DataSource should have either 2 or 4 subsets, but have {len(dsource.splits)}")

In [83]:
dbunch

In [84]:
from local.dicom2tensor import _plot_voxel

In [85]:
dsource.valid.items

[PosixPath('/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.21988995044.484726397.288.728'),
 PosixPath('/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.21988995044.484726615.348.2047'),
 PosixPath('/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.21988995044.484726680.961.2451'),
 PosixPath('/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.21988995044.484726770.896.3077'),
 PosixPath('/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.21988995044.484726854.615.3528'),
 PosixPath('/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.21988995044.484727037.584.4741'),
 PosixPath('/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.21988995044.484727132.597.5398'),
 PosixPath('/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.21988995044.484728039.807.6826'),
 PosixPath('/home/turgutluk/data/ventricles_data/mr/2.16.840.1.114362.1.11741058.

In [102]:
xs, ys = zip(*dsource.train)

In [ ]:
torch.unique(torch.stack(xs))

In [ ]:
torch.unique(torch.stack(ys))

In [86]:
# _plot_voxel(dsource.valid[0][0][0])

# _plot_voxel(dsource.valid[0][1])

### model 

In [87]:
from fastai2.torch_core import *
from fastai2.basics import *
from local.trainutils import *

In [88]:
# model
m = experiment_model_dict[model_name]()
apply_leaf(m, partial(my_cond_init, func=nn.init.kaiming_normal_))

### callbacks 

> There is an issue with CSVLogger - not able to create directory when not existing

> SaveModelCallback have problem with loading with distributed

In [89]:
from fastai2.callback.all import *

In [90]:
#export
class SaveModelCallback(TrackerCallback):
    "A `TrackerCallback` that saves the model's best during training and loads it at the end."
    def __init__(self, monitor='valid_loss', comp=None, min_delta=0., fname='model', every_epoch=False, add_save=None, with_opt=False):
        super().__init__(monitor=monitor, comp=comp, min_delta=min_delta)
        store_attr(self, 'fname,every_epoch,add_save,with_opt')

    def _save(self, name):
        self.learn.save(name, with_opt=self.with_opt)
        if self.add_save is not None:
            with self.add_save.open('wb') as f: self.learn.save(f, with_opt=self.with_opt)

    def after_epoch(self):
        "Compare the value monitored to its best score and save if best."
        if self.every_epoch: self._save(f'{self.fname}_{self.epoch}')
        else: #every improvement
            super().after_epoch()
            if self.new_best: self._save(f'{self.fname}')

# loading is a problem in distributed
#     def on_train_end(self, **kwargs):
#         "Load the best model."
#         if not self.every_epoch: self.learn.load(f'{self.fname}')

In [91]:
early_stop_cb = partial(EarlyStoppingCallback, monitor='dice_score', comp=np.greater, patience=5)
save_model_cb = partial(SaveModelCallback, monitor='dice_score', comp=np.greater, every_epoch=False,
                        fname=f'best_of_{MODEL_NAME}')
# csv_logger_cb = CSVLogger(fname=f'{experiment_id}_history.csv')

In [92]:
if early_stop: 
    if not int(gpu): print('early stop=True')
    cb_funcs = [early_stop_cb, save_model_cb]
else: 
    if not int(gpu): print('early stop=False')
    cb_funcs = [save_model_cb]
callbacks = [TerminateOnNaNCallback()]    

early stop=False


### learner

> Best models are saved under `experiments/{model_dir}/{MODEL_NAME}` directories

In [93]:
model_dir, MODEL_NAME

('atlas_brain_mr_models', 'ATLAS_Brain_MR_Baseline_1')

In [94]:
lf = loss_dict[loss_func]
opt_func = partial(Adam, eps=eps)
learn = Learner(dbunch, m, lf, metrics=[dice_score], opt_func=opt_func,
                path=Path('experiments')/model_dir,
                model_dir=MODEL_NAME, cbs=callbacks, cb_funcs=cb_funcs)
learn.to_fp16(dynamic=True)

In [95]:
learn.path, learn.model_dir

(PosixPath('experiments/atlas_brain_mr_models'), 'ATLAS_Brain_MR_Baseline_1')

### fit

In [96]:
epochs, lr

(1, 0.1)

In [97]:
opt_func.func

<function fastai2.optimizer.Adam(params, lr, mom=0.9, sqr_mom=0.99, eps=1e-05, wd=0.0, decouple_wd=True)>

In [98]:
learn.fit_one_cycle(epochs, lr_max=lr)

IndexError: list index out of range

### evaluate

In [32]:
f"test_results/{model_dir}/{MODEL_NAME}/{str(int(time()))}.txt"

'test_results/notl_brain_mr_models/NOTL_Brain_MR_Baseline_1/1575096711.txt'

In [37]:
learn.cbs = [cb for cb in learn.cbs if not isinstance(cb, TrackerCallback)]

In [39]:
# save test1 and test2 results for unique experiment at time.time()
if len(dbunch.dls) == 4: 
    test1_eval, test2_eval = learn.validate(2), learn.validate(3)
    eval_dir = f"test_results/{model_dir}/{MODEL_NAME}"
    os.makedirs(eval_dir, exist_ok=True)
    save_fn = f"{eval_dir}/{str(int(time()))}.txt"
    with open(save_fn, 'w') as f: f.write(str([test1_eval, test2_eval]))

### script

In [ ]:
data_name = "notl_brain_mr"
sample_size = 10
seed=42
bs=4
model_name = 'baseline1'
MODEL_NAME='NOTL_Brain_MR_Baseline_1'
model_dir='experiments/models/notl_brain_mr_models'
early_stop=False
loss_func='dice'
eps=1e-8
epochs=2
lr=0.1

In [ ]:
#export 
@call_parse
def main(
    gpu:Param("GPU to run on", str)=None,
    data_name:Param("Data name for experiment", str)="notl_brain_mr",
    sample_size:Param("Random samples for training, default None - full", int)=None,
    seed:Param("Random seed for sample_size", int)=None,
    bs:Param("Batch size for training", int)=4,
    model_name:Param("Model architecture config - baseline*", str)="baseline1",
    MODEL_NAME:Param("Model name to save the model", str)="NOTL_Brain_MR_Baseline_1",
    model_dir:Param("Directory to save model", str)="notl_brain_mr_models",
    early_stop:Param("Whether to use early stopping", bool)=False,
    loss_func:Param("Loss function for training", str)='dice',
    eps:Param("Eps value for Adam optimizer", float)=1e-8,
    epochs:Param("Number of epochs for training", int)=2,
    lr:Param("Learning rate for training", float)=0.1):
    
    "Distributed de novo training - aka. from scratch"
    import os; print(os.getcwd())

    gpu = setup_distrib(gpu)
    n_gpus, gpu_rank = num_distrib(), rank_distrib()

    # data
    dsource = datasource_dict[data_name]()
    if sample_size:
        with np_local_seed(seed):
            dsource.splits[0] = L(np.random.choice(dsource.splits[0], sample_size))  # subsample training
    dbunch = dsource.databunch(after_batch=[Cuda()], bs=bs)
    if len(dbunch.dls) == 4: pass
    elif len(dbunch.dls) == 2: pass
    else: raise Exception(f"DataSource should have either 2 or 4 subsets, but have {len(dsource.splits)}")

    # model
    m = experiment_model_dict[model_name]()
    apply_leaf(m, partial(my_cond_init, func=nn.init.kaiming_normal_))
    
    # callbacks
    save_model_cb = SaveModelCallback(monitor='dice_score', comp=np.greater, every_epoch=False,
                        fname=f'best_of_{MODEL_NAME}')
    callbacks = [TerminateOnNaNCallback(), save_model_cb]        
    
    # learn
    lf = loss_dict[loss_func]
    opt_func = partial(Adam, eps=eps)
    learn = Learner(dbunch, m, lf, metrics=[dice_score], opt_func=opt_func,
                    path=Path('experiments')/model_dir,
                    model_dir=MODEL_NAME, cbs=callbacks)
    learn.to_fp16()

    # distributed
    if gpu is None:       learn.to_parallel()
    elif num_distrib()>1: learn.to_distributed(gpu)    
    
    # fit
    learn.fit_one_cycle(epochs, lr_max=lr)

    # evaluate
    learn.cbs = [cb for cb in learn.cbs if not isinstance(cb, TrackerCallback)]
    test1_eval, test2_eval = learn.validate(2), learn.validate(3)
    if not gpu_rank:
        if len(dbunch.dls) == 4: 
            eval_dir = f"test_results/{model_dir}/{MODEL_NAME}"
            os.makedirs(eval_dir, exist_ok=True)
            save_fn = f"{eval_dir}/{str(int(time()))}.txt"
            with open(save_fn, 'w') as f: f.write(str([test1_eval, test2_eval]))

### Export

In [7]:
# special export to fix for relative import problems
from local.notebook.export import *

In [8]:
def export_nb_to(src, dest):
    "export src notebook to dest"
    cells = read_nb(src)['cells']
    srcs = []
    for cell in cells: 
        src = cell['source']
        if src.startswith("#export"): srcs.append(src+"\n") 
        if src.startswith("# export"): srcs.append(src+"\n")
    res = "\n".join(srcs)
    with open(dest, "w") as f: f.write(res)

In [9]:
export_nb_to("3b) traindenovo.ipynb", "scripts/traindenovo.py")

### test: script

In [5]:
import fastai2
from local.scriptrunner import *

launch_file = f"{Path(fastai2.__file__).parent}/launch.py"
script = "./scripts/traindenovo.py "

In [9]:
cmd = f"""python {launch_file} \
--gpus=01234 {script} \
--data_name=atlas_brain_mr \
--sample_size=10 \
--seed=42 \
--bs=2 \
--model_name=baseline1 \
--MODEL_NAME=ATLAS_Brain_MR_Baseline_1 \
--model_dir=atlas_brain_mr_models \
--early_stop=False \
--loss_func=bce \
--eps=1e-4 \
--epochs=2 \
--lr=3e-3"""

In [10]:
cmd

'python /home/turgutluk/fastai2_fork/fastai2/launch.py --gpus=01234 ./scripts/traindenovo.py  --data_name=atlas_brain_mr --sample_size=10 --seed=42 --bs=2 --model_name=baseline1 --MODEL_NAME=ATLAS_Brain_MR_Baseline_1 --model_dir=atlas_brain_mr_models --early_stop=False --loss_func=bce --eps=1e-4 --epochs=2 --lr=3e-3'

In [11]:
run_command(cmd)

/home/turgutluk/Vent_Seg_Project/dev
/home/turgutluk/Vent_Seg_Project/dev
/home/turgutluk/Vent_Seg_Project/dev
/home/turgutluk/Vent_Seg_Project/dev
/home/turgutluk/Vent_Seg_Project/dev
epoch     train_loss  valid_loss  dice_score  time
0         nan         01:26


KeyboardInterrupt: 